In [1]:
import json
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import KFold

In [2]:
training= pd.read_csv("ABIDE_train_images_all.csv")
training

,SUB_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,PATH
0,51456,1,55.40,M,126.0,118.0,128.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,1,22.90,M,107.0,119.0,93.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,1,39.20,M,93.0,80.0,108.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,1,22.80,M,106.0,94.0,118.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,1,34.60,F,133.0,135.0,122.0,WASI,WASI,WASI,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...,...,...,...,...,...,...,...
1080,50624,1,11.08,F,90.0,91.0,89.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1081,50625,1,7.00,M,99.0,90.0,98.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1082,50626,1,11.08,M,61.0,66.0,60.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1083,50627,1,9.50,F,88.0,103.0,84.0,DAS_II_SA,DAS_II_SA,DAS_II_SA,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [3]:
training=training.drop(["DX_GROUP","AGE_AT_SCAN","FIQ_TEST_TYPE","VIQ_TEST_TYPE","PIQ_TEST_TYPE"],axis=1)

In [4]:
df_FIQ=training[training["FIQ"].notna()].reset_index(drop=True).drop(["VIQ","PIQ"],axis=1)
df_FIQ


,SUB_ID,SEX,FIQ,PATH
0,51456,M,126.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,M,107.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,M,93.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,M,106.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,F,133.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
1045,50624,F,90.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1046,50625,M,99.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1047,50626,M,61.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1048,50627,F,88.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [5]:
df_FIQ=df_FIQ[df_FIQ["FIQ"]>=0].reset_index(drop=True)
df_FIQ=df_FIQ[["SUB_ID","FIQ","SEX","PATH"]]
df_FIQ['FIQ']=df_FIQ["FIQ"].astype('int64')
df_FIQ


,SUB_ID,FIQ,SEX,PATH
0,51456,126,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,107,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,93,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,106,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,133,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
1009,50624,90,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1010,50625,99,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1011,50626,61,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1012,50627,88,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [6]:
df_VIQ=training[training["VIQ"].notna()].reset_index(drop=True).drop(["FIQ","PIQ"],axis=1)
df_VIQ

,SUB_ID,SEX,VIQ,PATH
0,51456,M,118.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,M,119.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,M,80.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,M,94.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,F,135.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
911,50624,F,91.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
912,50625,M,90.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
913,50626,M,66.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
914,50627,F,103.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [7]:
df_VIQ=df_VIQ[df_VIQ["VIQ"]>=0].reset_index(drop=True)
df_VIQ=df_VIQ[["SUB_ID","VIQ","SEX","PATH"]]
df_VIQ['VIQ']=df_VIQ["VIQ"].astype('int64')
df_VIQ

,SUB_ID,VIQ,SEX,PATH
0,51456,118,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,119,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,80,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,94,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,135,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
893,50624,91,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
894,50625,90,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
895,50626,66,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
896,50627,103,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [8]:
df_PIQ=training[training["PIQ"].notna()].reset_index(drop=True).drop(["FIQ","VIQ"],axis=1)
df_PIQ

,SUB_ID,SEX,PIQ,PATH
0,51456,M,128.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,M,93.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,M,108.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,M,118.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,F,122.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
929,50624,F,89.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
930,50625,M,98.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
931,50626,M,60.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...
932,50627,F,84.0,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [9]:
df_PIQ=df_PIQ[df_PIQ["PIQ"]>=0].reset_index(drop=True)
df_PIQ=df_PIQ[["SUB_ID","PIQ","SEX","PATH"]]
df_PIQ['PIQ']=df_PIQ["PIQ"].astype('int64')
df_PIQ
df_PIQ

,SUB_ID,PIQ,SEX,PATH
0,51456,128,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
1,51457,93,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
2,51458,108,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
3,51459,118,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
4,51460,122,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
...,...,...,...,...
907,50624,89,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...
908,50625,98,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
909,50626,60,M,/neuro/labs/grantlab/research/MRI_Predict_Age/...
910,50627,84,F,/neuro/labs/grantlab/research/MRI_Predict_Age/...


In [10]:
kf5=KFold(n_splits=5,random_state=1,shuffle=True)

In [12]:
i=0
for result in kf5.split(df_FIQ):
    train=df_FIQ.iloc[result[0]]
    print(len(train))
    test=df_FIQ.iloc[result[1]]
    print(len(test))
    #train.to_csv(r'folderlist/FIQ_train_fold_'+ str(i) +'.list',sep='\t',index=False,header=None)
    #test.to_csv(r'folderlist/FIQ_test_fold_'+ str(i) +'.list',sep='\t',index=False,header=None)
    i=i+1

811
203
811
203
811
203
811
203
812
202


In [22]:
i=0
for result in kf5.split(df_VIQ):
    train=df_VIQ.iloc[result[0]]
    test=df_VIQ.iloc[result[1]]
    train.to_csv(r'folderlist/VIQ_train_fold_'+ str(i) +'.list',sep='\t',index=False)
    test.to_csv(r'folderlist/VIQ_test_fold_'+ str(i) +'.list',sep='\t',index=False)
    i=i+1

In [23]:
i=0
for result in kf5.split(df_PIQ):
    train=df_PIQ.iloc[result[0]]
    test=df_PIQ.iloc[result[1]]
    train.to_csv(r'folderlist/PIQ_train_fold_'+ str(i) +'.list',sep='\t',index=False)
    test.to_csv(r'folderlist/PIQ_test_fold_'+ str(i) +'.list',sep='\t',index=False)
    i=i+1